In [ ]:
%pip install pyfluidsynth

In [ ]:
from os.path import expanduser
import tensormidi

filename = expanduser('~/dev/data/midi/bach/raw/catech7.mid')

midi = tensormidi.load(filename)

midi.shape, midi.dtype

In [ ]:
from IPython.display import Audio
import fluidsynth
import numpy as np

samplerate = 44100
synth = fluidsynth.Synth(samplerate=samplerate)
synth.sfload('/usr/share/sounds/sf2/FluidR3_GM.sf2')

audio = np.zeros((0,2), np.int16)

for m in midi:
    dt = m['dt'] / 1e6 # microseconds to seconds
    if dt:
        nsamp = int(samplerate * dt)
        chunk = synth.get_samples(nsamp).reshape(-1, 2)
        audio = np.concatenate((audio, chunk))

    chan = m['channel']
    k, v = m['key'], m['value']
    
    synth.program_change(chan, m['program'])
    
    if m['type'] == tensormidi.NOTE_ON:
        synth.noteon(chan, k, v)
    elif m['type'] == tensormidi.NOTE_OFF:
        synth.noteoff(chan, k)
    elif m['type'] == tensormidi.CONTROL:
        synth.cc(chan, k, v)

Audio(data=audio[:, 0], rate=samplerate)